In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, tx

### Exercise 1




#### Make [this test](/edit/session2/tx.py) pass: `tx.py:TxTest:test_verify_p2wsh`

In [ ]:
# Exercise 1

reload(tx)
run(tx.TxTest('test_verify_p2wsh'))

In [ ]:
# example for creating a p2wsh bech32 address
from ecc import S256Point
from helper import encode_bech32_checksum, encode_varstr, sha256
from script import Script
sec1_hex = '0375e00eb72e29da82b89367947f29ef34afb75e8654f6ea368e0acdfd92976b7c'
sec2_hex = '03a1b26313f430c4b15bb1fdce663207659d8cac749a0e53d70eff01874496feff'
sec3_hex = '03c96d495bfdd5ba4145e3e046fee45e84a8a48ad05bd8dbb395c011a32cf9f880'
sec1 = bytes.fromhex(sec1_hex)
sec2 = bytes.fromhex(sec2_hex)
sec3 = bytes.fromhex(sec3_hex)
witness_script = Script([0x52, sec1, sec2, sec3, 0x53, 0xae])  # 2-of-3 multisig
s256 = sha256(witness_script.raw_serialize())
s = Script([0, s256])
raw = s.raw_serialize()
bech32 = encode_bech32_checksum(raw, testnet=False)
print(bech32)

### Exercise 2
#### Create a testnet 2-of-2 multisig p2wsh bech32 address using your private key from the Session 0 and this SEC public key:
`031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e`

Fill in the spreadsheet with the address.


In [ ]:
# Exercise 2

from ecc import PrivateKey
from helper import encode_bech32_checksum, encode_varstr, hash256, little_endian_to_int, sha256
from script import Script
sec2 = bytes.fromhex('031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e')
# use the same passphrase from session 0
passphrase = b'<fill this in>'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret

# get the public key using the .point property

# get the compressed SEC format of the point

# create the WitnessScript 0x52 (OP_2), sec1, sec2, 0x52, 0xae (OP_CHECKMULTISIG)
witness_script = Script([0x52, sec1, sec2, 0x52, 0xae])
# get the sha256 of the raw serialization of the script
s256 = sha256(witness_script.raw_serialize())
# create another script of 0 (OP_0) and the hash
s = Script([0, s256])
# get the raw serialization of the second script you just made

# encode to bech32 using encode_bech32_checksum, remember testnet=True

# print the address


### Exercise 3




#### Make [this test](/edit/session2/script.py) pass: `script.py:ScriptTest:test_p2wsh_address`

In [ ]:
# Exercise 3

reload(script)
run(script.ScriptTest('test_p2wsh_address'))

In [ ]:
# Example for signing a p2wsh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from script import Script
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
sec1 = private_key.point.sec()
sec2 = bytes.fromhex('031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e')
raw_tx_hex = '01000000000101dc60d3cc9fbfbfeae68031ee987d674315e3ec6fa5a910d913d1d70cc24bf6a70000000000ffffffff014c400f0000000000160014092ab91b37b4182061d9c01199aaac029f89561f0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
witness_script = Script([0x52, sec1, sec2, 0x52, 0xae])
z = tx_obj.sig_hash_bip143(input_index, witness_script=witness_script)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
print(sig.hex())

### Exercise 4




#### Make [this test](/edit/session2/tx.py) pass: `tx.py:TxTest:test_sign_p2wsh_multisig`

In [ ]:
# Exercise 4

reload(tx)
run(tx.TxTest('test_sign_p2wsh_multisig'))

In [ ]:
# Example for creating a p2wsh multisig transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
sec1 = private_key.point.sec()
sec2 = bytes.fromhex('031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e')
witness_script = Script([0x52, sec1, sec2, 0x52, 0xae])
sig2 = bytes.fromhex('304402206f68a4c8731b1981fde3ae2f8ac3e21dbb42903853a3cecf18c30c31d36b510102207e5eba87f5d9134307d0f0cfb3eb065b18bf8a35df9a8b10538586328d1c7aa101')
prev_tx_hex = 'a7f64bc20cd7d113d910a9a56fece31543677d98ee3180e6eabfbf9fccd360dc'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 0
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = p2wpkh_script(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
sig1 = tx_obj.get_sig_p2wsh_multisig(0, private_key, witness_script)
tx_obj.finalize_p2wsh_multisig_input(0, [sig1, sig2], witness_script)
print(tx_obj.serialize().hex())

### Exercise 5

#### Create a p2wpkh spending transaction

You have been sent 0.05 testnet BTC. Send 0.03 to `tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au`
and the change back to your bech32 address.


In [ ]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from network import SimpleNode
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
passphrase = b'<fill this in>'
private_key = PrivateKey(little_endian_to_int(hash256(passphrase)))
prev_tx_hex = '<fill this in>'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = -1  # fill this in
fee = 500
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
target_amount = 3000000
# create the transaction input

# create an array of tx_outs

# decode the target address to get the hash160 of the address

# create the target script pubkey using p2wpkh_script

# add the target transaction output
tx_outs.append(TxOut(target_amount, target_script_pubkey))
# calculate the change amount, remember you were sent 5000000 sats

# calculate the hash160 for your private key

# create the change script pubkey using p2wpkh_script


# create the transaction with testnet=True and segwit=True

# sign the one input with your private key

# print the hex to see what it looks like


### Exercise 6




#### Make [this test](/edit/session2/tx.py) pass: `tx.py:TxTest:test_verify_p2sh_p2wpkh`

In [ ]:
# Exercise 6

reload(tx)
run(tx.TxTest('test_verify_p2sh_p2wpkh'))

In [ ]:
# Example of generating a p2sh-p2wpkh address
from ecc import S256Point
from helper import encode_base58_checksum, hash160, h160_to_p2sh_address
sec_hex = '02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d'
point = S256Point.parse(bytes.fromhex(sec_hex))
h160 = point.hash160()
redeem_script = bytes([0]) + encode_varstr(h160)
h160_p2sh = hash160(redeem_script)
address = h160_to_p2sh_address(h160_p2sh, testnet=False)
print(address)

### Exercise 7
#### Create a testnet p2sh-p2wpkh address with your private key.



In [ ]:
# Exercise 7

from ecc import PrivateKey
from helper import encode_varstr, h160_to_p2sh_address, hash160, hash256, little_endian_to_int
# use the same passphrase from session 0
passphrase = b'<fill this in>'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret

# get the public key using the .point property

# get the hash160 of the point

# the RedeemScript starts with the segwit version (0 or b'\x00)

# next, add the hash160 to the RedeemScript using encode_varstr

# perform a hash160 to get the hash160 of the redeem script

# encode to base58 using h160_to_p2sh_address, remember testnet=True

# print the address


### Exercise 8




#### Make [this test](/edit/session2/ecc.py) pass: `ecc.py:S256Test:test_p2sh_p2wpkh_address`

In [ ]:
# Exercise 8

reload(ecc)
run(ecc.S256Test('test_p2sh_p2wpkh_address'))

In [ ]:
# Example for signing a p2sh-p2wpkh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from script import Script
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
raw_tx_hex = '010000000001014e6b786f3cd70ab1ffd75caa6bb252c9888fdca9ca94d40fec24bec3e643d89e0000000000ffffffff014c400f0000000000160014401af0b57c7a4b7490c508a47d0747d03cf6ac2e0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
z = tx_obj.sig_hash_bip143(input_index, redeem_script=redeem_script)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
tx_in = tx_obj.tx_ins[input_index]
tx_in.witness = [sig, sec]
tx_in.script_sig = Script([redeem_script.raw_serialize()])
print(tx_obj.verify_input(input_index))

### Exercise 9




#### Make [this test](/edit/session2/tx.py) pass: `tx.py:TxTest:test_sign_p2sh_p2wpkh`

In [ ]:
# Exercise 9

reload(tx)
run(tx.TxTest('test_sign_p2sh_p2wpkh'))

In [ ]:
# Example for creating a p2sh-p2wpkh transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
prev_tx_hex = '6c14a8370da20c7de5ebf216ece3156e99e7d6070442d93b80cdc344b2e80867'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 1
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = p2wpkh_script(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
tx_obj.sign_input(0, private_key, redeem_script=redeem_script)
print(tx_obj.serialize().hex())